# Map visualization

## Importing modules and packages

Cartopy is a Python package designed for geospatial data processing in order to produce maps and other geospatial data analyses.

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import os

## Creating a map

Define a projection:

* Equidistant cylindrical projection: a simple projection which maps meridians to vertical straight lines of constant spacing (for meridional intervals of constant spacing), and circles of latitude to horizontal straight lines of constant spacing (for constant intervals of parallels).
* Lambert Conformal Conic projection: one of the best projections for middle latitudes with an east–west orientation. It can use a single latitude line as its point of contact (a tangent line), or the cone can intersect the earth's surface along two lines, called secants. Along these two lines there is no distortion, but distortion does occur as the distance from the secants increases.

![Lambert Conformal Conic](4-Maps/image002.gif)


In [ ]:
#Define an equidistant cylindrical projection
prj_ec  = ccrs.PlateCarree()

ax = plt.axes(projection=prj_ec)
ax.gridlines(draw_labels=True)
ax.coastlines()

plt.show()

In [ ]:
#Define a Lambert Conformal Conic projection
prj_lcc = ccrs.LambertConformal(central_longitude  = -45.0, 
                                central_latitude   = -40.0, 
                                standard_parallels = (-40,-40), 
                                cutoff             = 30
                               )

ax = plt.axes(projection=prj_lcc)
ax.gridlines(draw_labels=True)
ax.coastlines()

plt.show()

## Plotting on a map

We can plot a line between two points

In [ ]:
lons = [-170., 60.]
lats = [-40., 10.]

In [ ]:
ax = plt.axes(projection=prj_ec)
ax.gridlines(draw_labels=True)
ax.coastlines()

ax.plot(lons, lats,
        color='blue', 
        linestyle='--',
       )

plt.show()

In [ ]:
ax = plt.axes(projection=prj_lcc)
ax.gridlines(draw_labels=True)
ax.coastlines()

ax.plot(lons, lats,
        color='blue', 
        linestyle='--',
        transform=prj_ec,
       )

plt.show()

In [ ]:
P1, P2 = prj_lcc.transform_points(prj_ec,np.array(lons),np.array(lats))
x = [P1[0],P2[0]]
y = [P1[1],P2[1]]

In [ ]:
ax = plt.axes(projection=prj_lcc)
ax.gridlines(draw_labels=True)
ax.coastlines()

ax.plot(x, y,
        color='red', 
        linestyle='-',
       )

plt.show()

## Opening the output netCDF file

Define the path and the filename of the netCDF file and open it as a dataset:

In [ ]:
path   = "4-Maps"
fname  = "puyehue-2011-regional.res.nc"
ncfile = os.path.join(path,fname)
ds     = xr.open_dataset(ncfile)

Print fields and global attributes:

In [ ]:
print(ds.keys())

Opening another file, e.g., ERA5 data:

In [ ]:
path   = "4-Maps"
fname  = "puyehue-2011-global.era5pl.0p5deg.nc"
ncfile = os.path.join(path,fname)
ds_met = xr.open_dataset(ncfile)

In [ ]:
ds_met

## Selecting variables

Next, select the variable you are interested in. For instance, the `COL_MASS` variable here:

In [ ]:
var = ds.COL_MASS
print(var)

You can see that `var` has assigned coordinate variables (lon, lat, and time) and some attributes with information. For example, in this case the column mass is stored in grams per meter square ($g\,m^{-2}$).

In [ ]:
col_mass = var.sel(time='2011-6-8 18:00')

You can retrieve global attributes with:

In [ ]:
latmin = ds.attrs['LATMIN']
latmax = ds.attrs['LATMAX']
lonmin = ds.attrs['LONMIN']
lonmax = ds.attrs['LONMAX']

In [ ]:
z = ds_met.z.sel(time='2011-6-8 18:00', level=250)/9.81

## Contours on a map

In [ ]:
ax = plt.axes(projection=prj_ec)
ax.coastlines()
gl=ax.gridlines(draw_labels=True)
gl.ylabels_right = False
gl.xlabels_top   = False

col_mass.plot.contourf(ax     = ax, 
                       vmin   = 0, 
                       vmax   = 50,
                       levels = 15,
                       #cmap   = 'RdBu',
                      )

plt.show()

In [ ]:
z.plot.contour(ax=ax, levels=8)
ax.get_figure()

Use `set_extent` to zoom in:

In [ ]:
extent = [ lonmin, lonmax, latmin, latmax ]

ax.set_extent(extent, crs=prj_ec)
ax.get_figure()

In [ ]:
ax = plt.axes(projection=prj_lcc)
ax.coastlines()
ax.gridlines(draw_labels=True)

col_mass.plot.contourf(ax   = ax, 
                       vmin = 0, 
                       vmax = 50,
                       transform = prj_ec, 
                      )

plt.show()

In [ ]:
ax = plt.axes(projection=prj_lcc)
ax.coastlines()
ax.gridlines(draw_labels=True)

levels= np.logspace(-1,2,num=11)

col_mass.plot.contourf(ax        = ax, 
                       levels    = levels,
                       cmap      = 'viridis',
                       transform = prj_ec, 
                      )

plt.show()

In [ ]:
ax = plt.axes(projection=prj_lcc)
ax.coastlines()
ax.gridlines(draw_labels=True)

levels= np.logspace(-1,2,num=11)

col_mass.plot.contourf(ax        = ax, 
                       levels    = levels,
                       cmap      = 'viridis',
                       extend    = 'max',
                       transform = prj_ec, 
                      )

plt.show()

In [ ]:
z.plot.contour(ax=ax, 
               transform=prj_ec, 
               levels=8
              )
ax.get_figure()

In [ ]:
ax.set_extent(extent, crs=prj_ec)
ax.gridlines(draw_labels=True)
ax.get_figure()

## Contour Raster Mode
Raster contours are created by individually assigning colors to the elements of a 2D array of rectangular cells. 
Raster fill can be much faster.

In [ ]:
ax = plt.axes(projection=prj_ec)
ax.coastlines()
gl = ax.gridlines(draw_labels=True)
gl.ylabels_right = False
gl.xlabels_top   = False

col_mass.plot.pcolormesh(ax   = ax, 
                         vmin = 0.1, 
                         vmax = 100,
                        )

plt.show()

In [ ]:
#Creating a logarithmic scale colormap
import matplotlib.colors as colors
vmin  = 0.1
vmax  = 100
norms = colors.LogNorm(vmin=vmin, vmax=vmax)

#Remove negative a zeros values
col_mass.values[col_mass.values<=vmin] = vmin

In [ ]:
ax = plt.axes(projection=prj_ec)
ax.coastlines()
gl=ax.gridlines(draw_labels=True)
gl.ylabels_right = False
gl.xlabels_top   = False

col_mass.plot.pcolormesh(ax   = ax, 
                         norm = norms,
                         cmap = 'viridis',
                        )

plt.show()

For regular grids you can use `imshow` for a better performance:

In [ ]:
ax = plt.axes(projection=prj_ec)
ax.coastlines()
gl=ax.gridlines(draw_labels=True)
gl.ylabels_right = False
gl.xlabels_top   = False

c=col_mass.plot.imshow(ax   = ax, 
                       norm = norms,
                       cmap = 'viridis',
                      )

#Zoom in
#ax.set_extent([-60,-59,-42,-41], crs=prj_ec)

plt.show()

## Performance

In [ ]:
%%time

ax = plt.axes(projection=prj_ec)
ax.coastlines()
ax.gridlines(draw_labels=True)

path = "4-Maps/native"
show_colorbar = True
for i in range(len(ds.time)):
    col_mass = var.isel(time=i)
    contour  = col_mass.plot.contourf(ax=ax, 
                                      vmin=0, 
                                      vmax=50,
                                      add_colorbar=show_colorbar,
                                     )
    show_colorbar = False
    fname = "{timestep:03d}.png".format(timestep=i)
    plt.savefig(os.path.join(path,fname))
    for coll in contour.collections:
        coll.remove()
    del contour

In [ ]:
%%time

ax = plt.axes(projection=prj_lcc)
ax.coastlines()
ax.gridlines(draw_labels=True)

path = "4-Maps/lambert"
show_colorbar = True
for i in range(len(ds.time)):
    col_mass = var.isel(time=i)
    contour  = col_mass.plot.contourf(ax=ax, 
                                      transform=prj_ec, 
                                      vmin=0, 
                                      vmax=50,
                                      add_colorbar=show_colorbar,
                                     )
    show_colorbar = False
    fname = "{timestep:03d}.png".format(timestep=i)
    plt.savefig(os.path.join(path,fname))
    for coll in contour.collections:
        coll.remove()
    del contour

In [ ]:
%%time

ax = plt.axes(projection=prj_ec)
ax.coastlines()
ax.gridlines(draw_labels=True)

path = "4-Maps/raster"
show_colorbar = True
for i in range(len(ds.time)):
    col_mass = var.isel(time=i)
    contour  = col_mass.plot.pcolormesh(ax=ax, 
                                        vmin=0, 
                                        vmax=50,
                                        add_colorbar=show_colorbar,
                                       )
    show_colorbar = False
    fname = "{timestep:03d}.png".format(timestep=i)
    plt.savefig(os.path.join(path,fname))
    contour.remove()
    del contour

In [ ]:
%%time

ax = plt.axes(projection=prj_ec)
ax.coastlines()
ax.gridlines(draw_labels=True)

path = "4-Maps/imshow"
show_colorbar = True
for i in range(len(ds.time)):
    col_mass = var.isel(time=i)
    contour  = col_mass.plot.imshow(ax=ax, 
                                    vmin=0, 
                                    vmax=50,
                                    add_colorbar=show_colorbar,
                                   )
    show_colorbar = False
    fname = "{timestep:03d}.png".format(timestep=i)
    plt.savefig(os.path.join(path,fname))
    contour.remove()
    del contour

## Closing files

In [ ]:
ds.close()

In [ ]:
ds_met.close()